# K-12 Diversity Statements Scraper

In [1]:
# Data manipulation libraries
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
# Common webscraping libaries
from bs4 import BeautifulSoup as bs
import requests

tqdm.pandas()

### Get list of universities

In [2]:
input_path = "../data/input/Fortune500_Diversity.xlsx"
uni_list = pd.read_excel(input_path)
uni_list = uni_list.melt(id_vars=['Name'], 
              value_vars=[x for x in uni_list.columns if "Unnamed" in x or "DIVERSITY" in x], 
              var_name='url_type', value_name='url')
print(uni_list.shape)
uni_list = uni_list.dropna(subset=["url"])
print(uni_list.shape)
uni_list = uni_list.drop_duplicates(subset="url")
print(uni_list.shape)
uni_list

(2100, 3)
(469, 3)
(466, 3)


,Name,url_type,url
0,Wal-Mart Stores,DIVERSITY PAGE,https://corporate.walmart.com/global-responsib...
1,Amazon.com,DIVERSITY PAGE,https://www.aboutamazon.com/workplace/diversit...
2,Apple,DIVERSITY PAGE,https://www.apple.com/diversity/
3,CVS Health,DIVERSITY PAGE,https://cvshealth.com/about/diversity
4,UnitedHealth Group,DIVERSITY PAGE,https://careers.unitedhealthgroup.com/culture/...
...,...,...,...
1249,Tenneco,Unnamed: 4,https://www.lb.com/responsibility/inclusion/ou...
1256,Principal Financial,Unnamed: 4,https://www.principal.com/about-us/corporate-c...
1393,Alleghany,Unnamed: 4,https://www.53.com/content/fifth-third/en/pers...
1687,U.S. Bancorp,Unnamed: 5,https://www.usfoods.com/careers/diversity/supp...


In [3]:
from utils import scrape_webpage
uni_list['text'] = uni_list.progress_apply(lambda x : scrape_webpage(x["url"],
                                                            x["Name"],
                                                            x["url_type"],
                                                            out_folder="fortune500_output",
                                                            save_images=False,
                                                            v=False),axis=1)
uni_list['failed_scrape'] = uni_list['text'].str.len() < 100
uni_list = uni_list[['Name','url_type','url','failed_scrape','text']]

  0%|          | 0/466 [00:00<?, ?it/s]

Energy Transfer Equity ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
WestRock HTTPSConnectionPool(host='www.westrock.com', port=443): Max retries exceeded with url: /en/company (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)')))
L Brands HTTPSConnectionPool(host='www.lb.com', port=443): Max retries exceeded with url: /responsibility/inclusion/overview (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)')))
Ball HTTPSConnectionPool(host='www.ball.com', port=443): Max retries exceeded with url: /na/about-ball/careers/diversity-inclusion (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)')))
Textron HTTPSConnectionPool(host='www

In [4]:
import re
# uni_list['text'] = uni_list['text'].map(lambda x : re.sub(r'[^ \w+]', '', x))
uni_list.to_csv(f"../data/fortune500_output/fortune500_diversity_statements.csv")